In [1]:
import numpy as np
from pyspark import SparkConf, SparkContext
import time
import re

In [2]:
app_name = "hw2_q1"
master_URL = "spark://192.168.35.128:7077"

In [3]:
start = time.time()
conf = SparkConf().setAppName(app_name).setMaster(master_URL)
sc = SparkContext(conf = conf)

In [4]:
# 將資料以"分隔
def deal_process(data):
    header = data.first()
    data = data.filter(lambda x: x!=header)
    data = data.map(lambda x: x.split('","'))
    return data

In [5]:
# 寫檔
def write(file_name, rdd_data, todo):
    output = open(file_name, "w")
    if todo == "time":
        header = '"Date", Word, Count\n'
        output.write(header)
        output_format = '"{}","{}",{}\n'
        for row in rdd_data.collect():
            output.write(output_format.format(row[0][0], row[0][1], row[1]))
    else:
        header = '"Word",Count\n'
        output.write(header)
        output_format = '"{}",{}\n'
        for row in rdd_data.collect():
            output.write(output_format.format(row[0], row[1]))

In [6]:
# 將資料進行預處理，以","分隔
data = deal_process(sc.textFile("News_Final.csv"))

In [7]:
# 全部資料的資料處理
data_title = data.map(lambda x: x[0].split(",\"")).flatMap(lambda x: re.split('[,* *]*', x[1]))
data_headline = data.flatMap(lambda x: re.split('[,* *]*', x[1]))
all_counts_of_title = data_title.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b).sortBy(lambda x: x[1], False).filter(lambda x: x[0] != "")
all_counts_of_headline = data_headline.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b).sortBy(lambda x: x[1], False).filter(lambda x: x[0] != "")
write("Title_frequent_words_in_total.csv", all_counts_of_title, "all")
write("Headline_frequent_words_in_total.csv", all_counts_of_headline, "all")

In [8]:
# topic為obama的資料處理
data_obama_title = data.filter(lambda x: x[3] == "obama").flatMap(lambda x: re.split('[,* *]*', x[0].split(",\"")[1]))
data_obama_headline = data.filter(lambda x: x[3] == "obama").flatMap(lambda x: re.split('[,* *]*', x[1]))
counts_of_obama_title = data_obama_title.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b).sortBy(lambda x: x[1], False).filter(lambda x: x[0] != "")
counts_of_obama_headline = data_obama_headline.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b).sortBy(lambda x: x[1], False).filter(lambda x: x[0] != "")
write("Title_frequent_words_in_obama.csv", counts_of_obama_title, "topic")
write("Headline_frequent_words_in_obama.csv", counts_of_obama_headline, "topic")

In [9]:
# topic為economy的資料處理
data_economy_title = data.filter(lambda x: x[3] == "economy").flatMap(lambda x: re.split('[,* *]*', x[0].split(",\"")[1]))
data_economy_headline = data.filter(lambda x: x[3] == "economy").flatMap(lambda x: re.split('[,* *]*', x[1]))
counts_of_economy_title = data_economy_title.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b).sortBy(lambda x: x[1], False).filter(lambda x: x[0] != "")
counts_of_economy_headline = data_economy_headline.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b).sortBy(lambda x: x[1], False).filter(lambda x: x[0] != "")
write("Title_frequent_words_in_economy.csv", counts_of_economy_title, "topic")
write("Headline_frequent_words_in_economy.csv", counts_of_economy_headline, "topic")

In [10]:
# topic為microsoft的資料處理
data_microsoft_title = data.filter(lambda x: x[3] == "microsoft").flatMap(lambda x: re.split('[,* *]*', x[0].split(",\"")[1]))
data_microsoft_headline = data.filter(lambda x: x[3] == "microsoft").flatMap(lambda x: re.split('[,* *]*', x[1]))
counts_of_microsoft_title = data_microsoft_title.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b).sortBy(lambda x: x[1], False).filter(lambda x: x[0] != "")
counts_of_microsoft_headline = data_microsoft_headline.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b).sortBy(lambda x: x[1], False).filter(lambda x: x[0] != "")
write("Title_frequent_words_in_microsoft.csv", counts_of_microsoft_title, "topic")
write("Headline_frequent_words_in_microsoft.csv", counts_of_microsoft_headline, "topic")

In [11]:
# topic為palestine的資料處理
data_palestine_title = data.filter(lambda x: x[3] == "palestine").flatMap(lambda x: re.split('[,* *]*', x[0].split(",\"")[1]))
data_palestine_headline = data.filter(lambda x: x[3] == "palestine").flatMap(lambda x: re.split('[,* *]*', x[1]))
counts_of_palestine_title = data_palestine_title.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b).sortBy(lambda x: x[1], False).filter(lambda x: x[0] != "")
counts_of_palestine_headline = data_palestine_headline.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b).sortBy(lambda x: x[1], False).filter(lambda x: x[0] != "")
write("Title_frequent_words_in_palestine.csv", counts_of_palestine_title, "topic")
write("Headline_frequent_words_in_palestine.csv", counts_of_palestine_headline, "topic")

In [12]:
# 以時間歸類的資料處理
title_time_data = data.map(lambda x: (x[4].split(" ")[0], re.split('[,* *]*', x[0].split(",\"")[1]))).flatMap(lambda x: [(x[0], element) for element in x[1]]).map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b).sortBy(lambda x: (x[0][0], x[1]), False).filter(lambda x: x[0][1] != "")
headline_time_data = data.map(lambda x: (x[4].split(" ")[0], re.split('[,* *]*', x[1]))).flatMap(lambda x: [(x[0], element) for element in x[1]]).map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b).sortBy(lambda x: (x[0][0], x[1]), False).filter(lambda x: x[0][1] != "")
write("Title_frequent_words_per_day.csv", title_time_data, "time")
write("Headline_frequent_words_per_day.csv", headline_time_data, "time")

In [13]:
print("time is {}".format(time.time() - start))

time is 222.89621543884277


In [14]:
sc.stop()